# DATA PROCESSING

El objetivo de este notebook es cargar cada uno de los archivos csv que contienen los datos de los diferentes datasets y realizar un preprocesamiento de los mismos para posteriormente combinarlos en un único dataFrame.

En el preprocesamiento se realizarán las siguientes tareas:
- Cargar los datos en un dataFrame
- Eliminar las columnas o filas de datos totales
- Eliminar los datos a nivel nacional
- Eliminar las columnas con el nombre de la comunidad autónoma para evitar duplicidad.

## Preparación del entorno

In [ ]:
import pandas as pd
import os 

RAW_DATA_PATH = os.path.join("..", "data/raw/")
SILVER_DATA_PATH = os.path.join("..", "data/silver/")

In [ ]:
comunidades_eliminar = ["Ceuta", "Melilla", "Canarias", "Balears, Illes"]
provincias_eliminar = ["Ceuta", "Melilla", "Palmas, Las", "Santa Cruz de Tenerife", "Balears, Illes"]

## Consumo Electrico Anual

In [ ]:
import pandas as pd

consumo_df = pd.read_csv(RAW_DATA_PATH+ "ConsumoElectricoAnual.csv", sep="\t", encoding="latin1")
consumo_df.rename(columns={"Comunidades y Ciudades Autï¿½nomas": "Comunidades Autónomas", "aï¿½o": "Año"}, inplace=True) #linux
consumo_df.rename(columns={"Comunidades y Ciudades Autónomas": "Comunidades Autónomas", "año": "Año"}, inplace=True) #windows
#nota: por alguna razón, el encoding funciona distinto, la función 'rename' ignora los errores si la columna no tiene el mismo nombre
consumo_df.info()

In [ ]:
consumo_df.head(7)

Comprobación de si hay datos nulos

In [ ]:
print(consumo_df.isnull().sum())

Eliminamos los valores nulos en las comunidades autónomas, que representan el total nacional.

In [ ]:
consumo_df.dropna(subset=["Comunidades Autónomas"], inplace=True)

Eliminamos las columnas "Total Nacional" y "Producto consumido", ya que, todos los registros contienen el mismo valor en ambas columnas

In [ ]:
consumo_df.drop(columns=["Total Nacional", "Producto consumido"], inplace=True)

Para cada valor de la columna "Comunidades y Ciudades Autónomas", eliminamos los 3 primeros carácteres

In [ ]:
consumo_df["Comunidades Autónomas"] = consumo_df["Comunidades Autónomas"].str.replace(r"\b\d{2} ", "", regex=True)

In [ ]:
consumo_df.head()

Transformamos el dataframe para que cada fila sea una comunidad autónoma y cada columna un año, en cada celda estará el valor total. También eliminamos las comunidades que no están en la península.

In [ ]:
consumo_df.set_index("Comunidades Autónomas", inplace=True)
consumo_df = consumo_df[~consumo_df.index.isin(comunidades_eliminar)]

In [ ]:
with open(SILVER_DATA_PATH + "consumo_electrico.csv", "w") as f:
    consumo_df.to_csv(f, sep=';', encoding='utf-8')

## Dispositivos de energía renovable

In [ ]:
dispositivos_df = pd.read_csv(RAW_DATA_PATH + "Dispositivos de energía renovable.csv", sep=";", encoding="latin")
dispositivos_df.info()

In [ ]:
dispositivos_df.rename(columns={"Nivel de ingresos mensuales netos del hogar": "Ingresos netos", "Tipo de dispositivo de energía renovable": "Tipo dispositivo"}, inplace=True)

In [ ]:
dispositivos_df.head()

Comprobamos que no haya valores nules ni columnas que no nos aporten información relevante


In [ ]:
print(dispositivos_df.isnull().sum())

In [ ]:
print(dispositivos_df.nunique())

Eliminamos los valores de Provincias que sean el Total Nacional ya que no vamos a necesitarlo

In [ ]:
dispositivos_df.drop(dispositivos_df[dispositivos_df["Provincias"] == "Total Nacional"].index, inplace=True)
dispositivos_df.head()

In [ ]:
dispositivos_df.drop(index = dispositivos_df[dispositivos_df["Tamaño del municipio"] != "Total"].index, inplace=True)
dispositivos_df.drop(columns=["Tamaño del municipio"], inplace=True)

In [ ]:
dispositivos_df = dispositivos_df[dispositivos_df['Ingresos netos'] != "Total"]

In [ ]:
dispositivos_df.head()

In [ ]:
dispositivos_df = dispositivos_df[dispositivos_df["Tipo dispositivo"].isin(["Total", "No dispone de dispositivo de energía renovable"])]

In [ ]:
df = dispositivos_df.copy()
df.set_index("Provincias", inplace=True)

In [ ]:
df_no_dispone = df[df["Tipo dispositivo"] == "No dispone de dispositivo de energía renovable"].copy()
df_no_dispone["Tipo dispositivo"] = "No"
df_total = df[df["Tipo dispositivo"] == "Total"].copy()

# Convertir a valor numerico
df_total["Total"] = df_total["Total"].str.replace(".", "").astype(int)
df_no_dispone["Total"] = df_no_dispone["Total"].str.replace(".", "").astype(int)

# Calcular "SI" = total - no
df_si_dispone = df_total.copy()
df_si_dispone["Tipo dispositivo"] = "Si"
df_si_dispone["Total"] = df_total["Total"] - df_no_dispone["Total"]
# Concatenar los datos
df_dispositivos = pd.concat([df_no_dispone, df_si_dispone]).sort_values(by=["Provincias","Ingresos netos", "Tipo dispositivo"])
df_dispositivos.rename(columns={"Tipo dispositivo": "Dispone de dispositivo de energía renovable"}, inplace=True)
df_dispositivos.head()

Eliminamos las provincias que no están en la península.

In [ ]:
df_dispositivos = df_dispositivos[~df_dispositivos.index.isin(provincias_eliminar)]

In [ ]:
with open(SILVER_DATA_PATH + "dispositivos_renovable.csv", "w") as f:
    df_dispositivos.to_csv(f, sep=';', encoding='utf-8')

## Distribución de edad de la población española

In [ ]:
edad_df=pd.read_csv(RAW_DATA_PATH+"Distribución de edad de la población española.csv", sep = ";", encoding = "latin")

edad_df.info()

In [ ]:
# ejemplo de los datos de la tabla para una provincia
edad_df.head(600).tail(8)

In [ ]:
print(edad_df[['Total Nacional', 'Españoles/Extranjeros','Sexo']].nunique())

Observamos que las columnas 'Total Nacional', 'Españoles/Extranjeros' y 'Sexo' no aportan ninguna información, por lo que se pueden eliminar.

In [ ]:
edad_df.drop(columns=['Total Nacional', 'Españoles/Extranjeros', 'Sexo'], inplace=True)

Hemos decidido quedarnos solo con los datos del 2022, por lo que se pueden eliminar las filas de otros años y después la columna 'Periodo'.

In [ ]:
edad_df.drop(edad_df[edad_df['Periodo'] != '1 de enero de 2022'].index, inplace=True)
edad_df.drop(columns=['Periodo'], inplace=True)

Los primeros registros del dataset son los datos a nivel nacional, donde la comunidad autónoma y la provincia son valores nulos, por lo que se pueden eliminar. Además, se eliminará la columna 'Comunidades y Ciudades Autónomas' y las filas con los totales en 'Edad'.

In [ ]:
edad_df.head()

In [ ]:
edad_df.dropna(subset=["Comunidades y Ciudades Autónomas", "Provincias"], inplace=True)
edad_df.drop(columns=['Comunidades y Ciudades Autónomas'], inplace=True)
edad_df.drop(edad_df[edad_df['Edad (grupos quinquenales)'] == 'Todas las edades'].index, inplace=True)

Se eliminará el número que identifica a cada provincia para dejar solo el nombre.

In [ ]:
edad_df["Provincias"] = edad_df["Provincias"].str.replace(r"^\d+\s", "", regex=True)

Una vez finalizado el preprocesamiento, el dataset queda con 1092 entradas y 3 columnas.

In [ ]:
edad_df.head()

In [ ]:
edad_df.info()

Transformamos el dataframe para que cada fila sea una comunidad autónoma y cada columna un año, en cada celda estará el valor total. También eliminamos las provincias que no están en la península.

In [ ]:
edad_df.set_index("Provincias", inplace=True)
edad_df = edad_df[~edad_df.index.isin(provincias_eliminar)]

In [ ]:
with open(SILVER_DATA_PATH + "distribucion_edad.csv", "w") as f:
    edad_df.to_csv(f, sep=';', encoding='utf-8')

## Intensidad de uso en viviendas

In [ ]:
import pandas as pd 

intensidad_df = pd.read_csv(RAW_DATA_PATH + "Intensidad de uso en viviendas.csv", sep=";", encoding="latin1")
intensidad_df.rename(columns={"Comunidades y Ciudades Autónomas": "Comunidades Autónomas"}, inplace=True)
intensidad_df.head()

Comprobamos si hay valores nulos en cada una de las columnas del dataframe

In [ ]:
intensidad_df.isnull().sum()

In [ ]:
len(intensidad_df["Comunidades Autónomas"].tolist())

En este proyecto se realizarán una serie de analísis de datos teniendo en cuenta únicamente las comunidades autónomas y las provincias, por lo que se elimina la columna "Municipios",ya que no nos ofrece información útil para el estudio.

Además, se elimina la columna "Total Nacional" ya que tampoco aporta ninguna información.

In [ ]:
intensidad_df.drop(columns=["Municipios","Total Nacional"], inplace=True)

Eliminamos las filas con valores en "Provincia" y "Comunidad Autónoma" nulos. Además de la columna Comunidad Autónoma.

In [ ]:
intensidad_df.dropna(subset=["Provincias"],inplace=True)
intensidad_df.drop(columns=["Comunidades Autónomas"], inplace=True)

Quitamos el número de la provincia para dejar solo el nombre.

In [ ]:
intensidad_df["Provincias"] = intensidad_df["Provincias"].str.replace(r"^\d+\s", "", regex=True)

Transformamos el dataframe para que cada fila sea una comunidad autónoma y cada columna un año, en cada celda estará el valor total. También eliminamos las provincias que no están en la península.

In [ ]:
intensidad_df.set_index("Provincias", inplace=True)
intensidad_df = intensidad_df[~intensidad_df.index.isin(provincias_eliminar)]

In [ ]:
with open(SILVER_DATA_PATH + "intensidad_de_uso_en_viviendas.csv", "w") as f:
    intensidad_df.to_csv(f, sep=';', encoding='utf-8')

## Producción de energía por lugar

In [ ]:
produccion_df = pd.read_csv(RAW_DATA_PATH + "Producción de energía por lugar.csv", sep=",", encoding="utf-8")
produccion_df.info()

In [ ]:
produccion_df.head()

Comporobamos si hay valores nulos o columnas que no nos aporten información

In [ ]:
print(produccion_df.isnull().sum())

In [ ]:
print(produccion_df.nunique())

Eliminamos los valores nulos de Potencia y de Provincia y Comunidad Autonoma los no encontrados

Eliminamos también las columnas Latitud y Longitud

In [ ]:
produccion_df.dropna(subset=["Potencia"], inplace=True)
produccion_df.drop(produccion_df[produccion_df["Provincia"] == "Provincia no encontrada"].index, inplace=True)
produccion_df.drop(produccion_df[produccion_df["Comunidad Autonoma"] == "Comunidad autónoma no encontrada"].index, inplace=True)
produccion_df.drop(columns=["Latitud", "Longitud"], inplace=True)

Separamos los valores de Potencia en Mes y Potencia kWh 

In [ ]:
produccion_df["Mes"] = produccion_df["Potencia"].str.extract(r'^PV output([^:]+)')
produccion_df["Potencia"] = produccion_df["Potencia"].str.extract(r":\s*([\d.]+)").astype(float)
produccion_df.rename(columns={"Potencia": "Potencia kWh"}, inplace=True)
produccion_df.head(24)

Como podemos observar que hay varios valores para la misma provincia, esto se debe a que el dataset lo divide por latitudes y longitudes. Hacemos un sumatorio para tener los valores totales por provincia 

In [ ]:
produccion_df["Potencia kWh"] = produccion_df.groupby(["Comunidad Autonoma", "Provincia", "Mes"])["Potencia kWh"].transform("mean")
produccion_df.drop_duplicates(subset=["Comunidad Autonoma", "Provincia", "Mes"], inplace=True)
produccion_df.head(24)

In [ ]:
produccion_df.drop(columns = ["Comunidad Autonoma"], inplace=True)

## Renta por hogar 

In [ ]:
renta_df=pd.read_csv(RAW_DATA_PATH+"Renta media por hogar.csv", sep = ";", encoding = "latin")

renta_df.info()

In [ ]:
# Ejemplo de los datos de la tabla para una provincia
renta_df.head(68).tail(8)

In [ ]:
#Eliminamos columna Islas, pues no es de interés.
renta_df.drop(columns=['Islas'], inplace=True)

#Eliminar la columna "Renta anual media por hogar" pues no es relevante.
renta_df.drop(columns='Indicadores de renta media', inplace=True)

#Eliminar la columna "Comunidades y Ciudades Autónomas".
renta_df.drop(columns='Comunidades y Ciudades Autónomas', inplace=True)

renta_df.head()

In [ ]:
#Como la provincia de Navarra en el año 2020 tiene un valor nulo, se añade su correspondiente valor, consultado en Nastat.
#Ver valor primero

# Localizamos la fila específica para Navarra en 2020 y le asignamos un nuevo valor en la columna "Total"
renta_df.loc[(renta_df['Provincias'] == 'Navarra') & (renta_df['Periodo'] == 2020), 'Total'] = 37082

renta_df.loc[(renta_df['Provincias'] == 'Navarra') & (renta_df['Periodo'] == 2020), 'Total']

In [ ]:
#Información actual de nuestro dataframe:
renta_df.info()

#Cómo se ve el dataframe ahora:
renta_df.head(8)

Transformamos el dataframe para que cada fila sea una comunidad autónoma y cada columna un año, en cada celda estará el valor total. También eliminamos las provincias que no están en la península.

In [ ]:
renta_df.set_index("Provincias", inplace=True)
renta_df = renta_df[~renta_df.index.isin(provincias_eliminar)]

In [ ]:
with open(SILVER_DATA_PATH + "renta_media_hogar.csv", "w") as f:
    renta_df.to_csv(f, sep=';', encoding='utf-8')

## Tipo de núcleo familiar

In [ ]:
tipo_nucleo_familiar_df=pd.read_csv(RAW_DATA_PATH+"Tipo de núcleo familiar.csv", sep = ";", encoding = "latin")

tipo_nucleo_familiar_df.info()

In [ ]:
# Ejemplo de los datos que presenta la tabla
tipo_nucleo_familiar_df.head(800).tail(10)

En la columna "Provincias", algunas filas están etiquetadas como "Total Nacional". Para simplificar el conjunto de datos, se ha decidido eliminar los registros correspondientes a los datos a nivel nacional.

In [ ]:
tipo_nucleo_familiar_df.head(5)

In [ ]:
tipo_nucleo_familiar_df = tipo_nucleo_familiar_df[tipo_nucleo_familiar_df['Provincias'] != "Total Nacional"]


In [ ]:
tipo_nucleo_familiar_df.head(10)

En la columna "Tipo de núcleo familiar" se ha encontrado un valor "Total", el cual será eliminado.

In [ ]:
tipo_nucleo_familiar_df = tipo_nucleo_familiar_df[tipo_nucleo_familiar_df['Tipo de núcleo familiar'] != "Total"]


Lo mismo sucede con la columna "Número de hijos".

In [ ]:
tipo_nucleo_familiar_df = tipo_nucleo_familiar_df[tipo_nucleo_familiar_df['Número de hijos'] != "Total"]

En el dataset, el símbolo '.' debe interpretarse como dato que no se proporciona por muestra insuficiente.

In [ ]:
columnas_a_verificar = ['Provincias', 'Tipo de núcleo familiar', 'Número de hijos', 'Total']
tipo_nucleo_familiar_df[columnas_a_verificar].apply(lambda x: x == '.').any()

No se han encontrado valores nulos.

Una vez finalizado el preprocesamiento, el dataset queda con 1248 entradas y 4 columnas.

In [ ]:
tipo_nucleo_familiar_df.head(10)


In [ ]:
a = tipo_nucleo_familiar_df['Tipo de núcleo familiar'].unique()
print(a)    

Transformamos el dataframe para que cada fila sea una comunidad autónoma y cada columna un año, en cada celda estará el valor total. También eliminamos las provincias que no están en la península.

In [ ]:
tipo_nucleo_familiar_df.set_index("Provincias", inplace=True)
tipo_nucleo_familiar_df = tipo_nucleo_familiar_df[~tipo_nucleo_familiar_df.index.isin(provincias_eliminar)]

In [ ]:
with open(SILVER_DATA_PATH + "tipo_nucleo_familiar.csv", "w") as f:
    tipo_nucleo_familiar_df.to_csv(f, sep=';', encoding='utf-8')

Transformamos el dataframe para que cada fila sea una comunidad autónoma y cada columna un año, en cada celda estará el valor total. También eliminamos las provincias que no están en la península.

In [ ]:
intensidad_df.set_index("Provincias", inplace=True)
intensidad_df = intensidad_df[~intensidad_df.index.isin(provincias_eliminar)]

In [ ]:
with open(SILVER_DATA_PATH + "intensidad_de_uso_en_viviendas.csv", "w") as f:
    intensidad_df.to_csv(f, sep=';', encoding='utf-8')

## Viviendas según el número de personas

In [ ]:
viviendas_numero_personas_df=pd.read_csv(RAW_DATA_PATH+"Viviendas según el número de personas.csv", sep = ";", encoding = "latin")

viviendas_numero_personas_df.info()

In [ ]:
# Ejemplo de los datos que presenta la tabla
viviendas_numero_personas_df.head(10)

In [ ]:
print(viviendas_numero_personas_df[['Total Nacional']].nunique())

La columna "Total Nacional" se eliminará ya que no añade valor al conjunto de datos.

In [ ]:
viviendas_numero_personas_df.drop(columns=['Total Nacional'], inplace=True)

Se van a eliminar las primeras 28 filas del dataset ya que corresponden a los datos totales nacionales.

In [ ]:
viviendas_numero_personas_df = viviendas_numero_personas_df.drop(index=range(0, 28))

Para cada comunidad autónoma, hay filas que corresponden al total de dicha comunidad, lo cual se representa con valores nulos en la columna "Provincias". El inconveniente es que, en comunidades con solo una provincia, los valores de la columna "Provincias" también son nulos. Por esta razón, se desea asignar el nombre de la provincia a las filas de comunidades autónomas que cuentan con una sola provincia y eliminar los valores totales en las comunidades autónomas con varias provincias.

In [ ]:
# Definir el diccionario de asignaciones
asignaciones = {
    "03 Asturias, Principado de": "Asturias",
    "04 Balears, Illes": "Balears, Illes",
    "06 Cantabria": "Cantabria",
    "13 Madrid, Comunidad de": "Murcia",
    "14 Murcia, Región de": "Madrid",
    "15 Navarra, Comunidad Foral de":"Navarra", 
    "17 Rioja, La": "Rioja, La",
    "18 Ceuta": "Ceuta",
    "19 Melilla": "Melilla"
}

# Identificar las comunidades autónomas con una sola provincia
comunidades_uniprovinciales = set(asignaciones.keys())

# Reemplazar valores nulos en "Provincias" para comunidades uniprovinciales
for comunidad, provincia in asignaciones.items():
    viviendas_numero_personas_df.loc[(viviendas_numero_personas_df["Comunidades y Ciudades Autónomas"] == comunidad) & (viviendas_numero_personas_df["Provincias"].isnull()), "Provincias"] = provincia

# Eliminar filas de valores totales por comunidad autónoma
viviendas_numero_personas_df = viviendas_numero_personas_df.dropna(subset=["Provincias"])


Se elimina la columna por comunidad autónoma.

In [ ]:
viviendas_numero_personas_df = viviendas_numero_personas_df.drop(columns=["Comunidades y Ciudades Autónomas"])

En la columna "Tamaño del hogar", hay filas que contienen el total y otras que indican el tamaño medio. Se eliminarán aquellas que cumplan con cualquiera de estas condiciones.

In [ ]:
viviendas_numero_personas_df = viviendas_numero_personas_df[viviendas_numero_personas_df['Tamaño del hogar'] != "Total"]
viviendas_numero_personas_df = viviendas_numero_personas_df[viviendas_numero_personas_df['Tamaño del hogar'] != "Tamaño medio"]

Hemos decidido quedarnos solo con los datos del 2022, por lo que se pueden eliminar las filas de otros años y después la columna 'Año'.

In [ ]:
viviendas_numero_personas_df.drop(viviendas_numero_personas_df[viviendas_numero_personas_df['Año'] != 2022].index, inplace=True)
viviendas_numero_personas_df.drop(columns=['Año'], inplace=True)

Una vez finalizado el preprocesamiento, el dataset queda con 260 entradas y 3 columnas.

In [ ]:
viviendas_numero_personas_df.info()

In [ ]:
viviendas_numero_personas_df.head(10)

Transformamos el dataframe para que cada fila sea una comunidad autónoma y cada columna un año, en cada celda estará el valor total. También eliminamos las provincias que no están en la península.

In [ ]:
viviendas_numero_personas_df.set_index("Provincias", inplace=True)
viviendas_numero_personas_df = viviendas_numero_personas_df[~viviendas_numero_personas_df.index.isin(provincias_eliminar)]

In [ ]:
with open(SILVER_DATA_PATH + "viviendas_personas.csv", "w") as f:
    viviendas_numero_personas_df.to_csv(f, sep=';', encoding='utf-8')